<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_05/class_005_feature_extraction_image_004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
from google.colab import files

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
try:
  import glrlm
except:
  !pip install pip glrlm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
try:
  import mahotas
except:
  !pip install mahotas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 4.3 MB/s 


In [5]:
import glrlm
from glrlm import GLRLM

In [6]:
import mahotas

In [7]:
# check https://pypi.org/project/glrlm/

In [8]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 139MB/s]


In [9]:
FILE='trash_nov_22_2018.zip'

In [10]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [11]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [12]:
Img_Size = 80
ref = 'jpg'
notref = 'met'

In [13]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [14]:
def new_names(letter,names):
  lista = []
  for x in names:
    lista.append(x+letter)
  return lista

In [15]:
# glcm features

def properties_glcm(img,Img_Size):
  glcm =[]
  glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']


features =[]

for i in range(len(img_name)):
  glcm = []
  glrlm_add = []
  img = X[i]
  glcm = properties_glcm(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  values_green = properties_glcm(green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties_glcm(red_img,Img_Size)


  blue_img = img[:,:,2]
  values_blue = properties_glcm(blue_img,Img_Size)


  total= [*glcm,*values_red, *values_green, *values_blue]

  features.append(total)

names = glcm_names + new_names('_r',glcm_names)
names = names + new_names('_g',glcm_names)
names = names + new_names('_b',glcm_names)

df_glcm = pd.DataFrame(features,columns=names)

In [16]:
Arq = "output_excel_glcm.xlsx"
df_glcm.to_excel(Arq)
files.download(Arq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
df_glcm

,contrast,dissimilarity,homogeneity,energy,correlation,ASM,contrast_r,dissimilarity_r,homogeneity_r,energy_r,...,homogeneity_g,energy_g,correlation_g,ASM_g,contrast_b,dissimilarity_b,homogeneity_b,energy_b,correlation_b,ASM_b
0,498.964103,9.867628,0.238681,0.029830,0.948122,0.000890,590.412821,11.158333,0.206623,0.029813,...,0.232813,0.030270,0.947509,0.000916,447.287660,9.174840,0.254093,0.028129,0.949031,0.000791
1,356.344391,11.277083,0.178445,0.027051,0.991291,0.000732,378.684455,11.681891,0.174064,0.026368,...,0.177230,0.027354,0.991109,0.000748,343.230128,11.041346,0.178819,0.027156,0.991610,0.000737
2,834.724359,12.536218,0.247208,0.028026,0.875504,0.000785,609.068109,11.054968,0.245956,0.028595,...,0.247567,0.028183,0.871604,0.000794,1018.333974,13.876923,0.237472,0.027138,0.890343,0.000736
3,854.156250,13.713301,0.399496,0.043164,0.899442,0.001863,892.051923,15.163782,0.370711,0.037676,...,0.392301,0.041822,0.900380,0.001749,865.250000,13.726603,0.384635,0.039732,0.910023,0.001579
4,194.098718,9.643910,0.158241,0.024129,0.984573,0.000582,194.019071,9.525801,0.158687,0.026792,...,0.156499,0.024323,0.985102,0.000592,186.454006,9.536378,0.156162,0.023437,0.983407,0.000549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522,402.962660,5.342788,0.592608,0.128217,0.935375,0.016439,292.621955,5.453686,0.551822,0.120676,...,0.563272,0.114210,0.926794,0.013044,437.137340,5.059936,0.602917,0.113037,0.950554,0.012777
2523,394.603846,5.141346,0.613960,0.125931,0.677457,0.015858,339.639744,5.405609,0.558249,0.117918,...,0.597848,0.115631,0.678537,0.013370,388.748397,5.103205,0.605563,0.118332,0.708303,0.014002
2524,549.683494,10.464263,0.442215,0.057080,0.951450,0.003258,665.495032,11.204006,0.417871,0.052755,...,0.420420,0.053648,0.958484,0.002878,497.388462,9.639744,0.406087,0.052600,0.906744,0.002767
2525,362.160897,8.441026,0.463003,0.054965,0.965576,0.003021,442.232372,9.282372,0.430610,0.048901,...,0.450668,0.053033,0.968534,0.002812,364.482051,8.501603,0.419595,0.054352,0.954865,0.002954


In [18]:
# glrlm features

glrlm_names =['SRE','LRE','GLU','RLU','RPC']

app = GLRLM()

def glrlm_features(img,Img_Size,model=app):
  resized = cv2.resize(img, (Img_Size,Img_Size))
  glrlm_get = model.get_features(resized, 8)
  return glrlm_get.Features

features =[]

for i in range(len(img_name)):

  glrlm_add = []
  img = X[i]
  glrlm_add = glrlm_features(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  glrlm_green = glrlm_features(green_img,Img_Size)

  red_img = img[:,:,0]
  glrlm_red = glrlm_features(red_img,Img_Size)

  blue_img = img[:,:,2]
  glrlm_blue = glrlm_features(blue_img,Img_Size)

  total= [*glrlm_add, *glrlm_red, *glrlm_green, *glrlm_blue]

  features.append(total)


names = glrlm_names + new_names('_r',glrlm_names)
names = names + new_names('_g',glrlm_names)
names = names + new_names('_b',glrlm_names)

df_glrlm = pd.DataFrame(features,columns=names)

In [19]:
Arq = "output_excel_glrlm.xlsx"
df_glrlm.to_excel(Arq)
files.download(Arq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df_glrlm

,SRE,LRE,GLU,RLU,RPC,SRE_r,LRE_r,GLU_r,RLU_r,RPC_r,SRE_g,LRE_g,GLU_g,RLU_g,RPC_g,SRE_b,LRE_b,GLU_b,RLU_b,RPC_b
0,1.885,441.593,601.989,1005.119,7.326,1.862,536.232,664.287,896.815,6.665,1.746,659.323,766.338,687.234,5.794,1.782,580.653,737.706,742.000,6.034
1,1.745,458.920,659.125,802.177,6.752,1.624,743.034,850.443,546.727,5.282,1.715,469.567,675.001,752.229,6.577,1.668,672.525,799.677,609.781,5.597
2,2.103,382.622,565.440,1318.246,7.882,2.090,352.974,551.256,1333.303,8.064,2.110,402.531,572.644,1312.092,7.785,2.109,252.945,472.342,1589.454,9.362
3,2.034,273.901,501.999,1385.442,8.814,2.041,294.817,493.519,1424.571,8.982,2.042,262.164,490.380,1424.687,9.009,2.093,354.629,531.776,1379.448,8.323
4,1.698,623.255,781.935,621.709,5.609,1.739,681.119,788.395,645.759,5.563,1.749,445.448,658.773,788.733,6.644,1.706,619.011,773.733,637.034,5.679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522,0.995,5024.940,2526.120,105.267,1.811,1.181,2959.171,1820.184,147.692,2.483,1.096,5082.390,2572.518,115.539,1.814,1.613,1252.314,1088.739,425.003,4.120
2523,1.599,4036.302,2248.371,209.435,1.973,1.778,1421.030,1138.229,468.518,3.878,1.625,3721.999,2108.603,221.793,2.101,1.801,1131.377,1035.175,527.994,4.241
2524,1.339,695.755,864.962,386.904,5.148,1.540,629.252,796.398,526.686,5.586,1.375,704.367,857.199,408.853,5.192,1.474,769.791,892.739,431.086,4.976
2525,1.320,674.114,877.712,377.511,5.061,1.354,528.355,748.103,474.507,5.944,1.220,666.681,878.607,338.730,5.048,1.293,766.157,957.665,328.023,4.638


In [21]:
# Local Binary Patterns features
# check https://mahotas.readthedocs.io/en/release-1.0/lbp.html

def properties_LBP(img,Img_Size):
  lbp_features =[]
 
  # adding gaussian filter
  img = mahotas.gaussian_filter(img, 4)
  
  # setting threshold
  threshed = (img > img.mean())
  
  # making is labelled image
  labeled, n = mahotas.label(threshed)
  
  resp = mahotas.features.lbp(labeled,200, 5)
  
  return resp


features =[]

for i in range(len(img_name)):

  img = X[i]
  LBP = properties_LBP(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  values_green = properties_LBP(green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties_LBP(red_img,Img_Size)


  blue_img = img[:,:,2]
  values_blue = properties_LBP(blue_img,Img_Size)


  total= [*LBP,*values_red, *values_green, *values_blue]

  features.append(total)


#df_glcm = pd.DataFrame(features,columns=names)
df_LBP = pd.DataFrame(features)
df_LBP.shape

(2527, 32)

In [22]:
Arq = "output_excel_LBP.xlsx"
df_LBP.to_excel(Arq)
files.download(Arq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
df_LBP

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,98507.0,72820.0,20929.0,0.0,4352.0,0.0,0.0,0.0,98715.0,73261.0,...,0.0,0.0,110805.0,60474.0,20202.0,0.0,5127.0,0.0,0.0,0.0
1,112047.0,49528.0,35033.0,0.0,0.0,0.0,0.0,0.0,110941.0,51552.0,...,0.0,0.0,111847.0,49488.0,35267.0,0.0,6.0,0.0,0.0,0.0
2,128064.0,35953.0,27690.0,781.0,4120.0,0.0,0.0,0.0,122667.0,42130.0,...,0.0,0.0,130756.0,32819.0,28654.0,834.0,3545.0,0.0,0.0,0.0
3,110335.0,54333.0,19188.0,5999.0,5496.0,465.0,792.0,0.0,108342.0,53490.0,...,788.0,0.0,112259.0,52506.0,21681.0,5739.0,4230.0,193.0,0.0,0.0
4,111833.0,57103.0,27672.0,0.0,0.0,0.0,0.0,0.0,122947.0,46375.0,...,0.0,0.0,109556.0,59115.0,27937.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522,157060.0,10110.0,17917.0,148.0,7497.0,2528.0,1312.0,36.0,147594.0,18323.0,...,119.0,0.0,167233.0,5677.0,7208.0,120.0,5216.0,1868.0,9048.0,238.0
2523,120144.0,43352.0,29066.0,694.0,2975.0,223.0,154.0,0.0,120184.0,42368.0,...,25.0,0.0,115925.0,46070.0,20442.0,2378.0,9680.0,1512.0,601.0,0.0
2524,111448.0,45225.0,15478.0,17205.0,2989.0,3759.0,504.0,0.0,123244.0,29248.0,...,1912.0,0.0,125716.0,41708.0,24894.0,2023.0,2141.0,126.0,0.0,0.0
2525,109654.0,51563.0,17806.0,12375.0,4919.0,291.0,0.0,0.0,129565.0,18132.0,...,0.0,0.0,115611.0,53089.0,19898.0,5174.0,2782.0,54.0,0.0,0.0
